In [1]:
pip install torch torchvision timm

Note: you may need to restart the kernel to use updated packages.


In [15]:
import torch
import torch.nn as nn
import timm
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class AlzheimerDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Example transformation pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
train_data_dir = r'C:\Users\Administrator\Desktop\FINAL YEAR PROJECT\Dataset\Modified DATASET\train'  # Update this to your actual train folder path

In [9]:
# For loading the training dataset
train_image_paths = []
train_labels = []
for label, class_dir in enumerate(['no impairment', 'very mild impairment', 'mild impairment', 'moderate impairment']):
    full_path = os.path.join(train_data_dir, class_dir)
    for file_name in os.listdir(full_path):
        train_image_paths.append(os.path.join(full_path, file_name))
        train_labels.append(label)

train_dataset = AlzheimerDataset(train_image_paths, train_labels, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [10]:
test_data_dir = r'C:\Users\Administrator\Desktop\FINAL YEAR PROJECT\Dataset\Modified DATASET\test'  # Update this to your actual test folder path

In [11]:
# For loading the testing dataset
test_image_paths = []
test_labels = []
for label, class_dir in enumerate(['no impairment', 'very mild impairment', 'mild impairment', 'moderate impairment']):
    full_path = os.path.join(test_data_dir, class_dir)
    for file_name in os.listdir(full_path):
        test_image_paths.append(os.path.join(full_path, file_name))
        test_labels.append(label)

test_dataset = AlzheimerDataset(test_image_paths, test_labels, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [16]:
class SwinAlzheimerModel(nn.Module):
    def __init__(self, num_classes=4):
        super(SwinAlzheimerModel, self).__init__()
        # Load pre-trained Swin Transformer from timm
        self.swin_transformer = timm.create_model('swin_base_patch4_window7_224', pretrained=True)
        # Replace classifier head with your custom head
        self.swin_transformer.head = nn.Linear(self.swin_transformer.head.in_features, num_classes)

    def forward(self, x):
        return self.swin_transformer(x)

model = SwinAlzheimerModel(num_classes=4).to(device)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
num_epochs = 25

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Accuracy: {train_accuracy:.2f}%")
    
    scheduler.step()

# Model evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Test Accuracy: {test_accuracy:.2f}%")

In [ ]:
torch.save(model.state_dict(), 'swin_transformer_alzheimer.pth')
print("Saved")

In [ ]:
def predict(image_path, model):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    
    return predicted.item()

# Example usage
prediction = predict('/path_to_mri_image', model)
print(f'Predicted class: {prediction}')

In [ ]:
def evaluate_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for efficiency
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            # Count the total and correctly predicted samples
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate overall accuracy
    accuracy = 100 * correct / total
    return accuracy

# Example usage
test_accuracy = evaluate_model(model, test_loader, device)
print(f"Exact Test Accuracy: {test_accuracy:.2f}%")